# Data Model Mover

**This tutorial shows how to copy a data model from one team/ data pool to another one, independent of the cluster.**



**To do so we first need to connect to the source data model.**

In [32]:
from pycelonis import get_celonis

c_source = get_celonis("URL to the team from which you want to copy.", "Specify a valid API key for your source cloud team.")
source_data_model = c_source.datamodels.find("ID of the source data model. It can be copied from the URL.") 

2020-02-13 15:59:50 - pycelonis: Login successful! Hello s.matthaei@celonis.com


**In the next step we connect to the target data pool.**

In [39]:
c_target = get_celonis("URL to cloud team where you want to copy to.", "Specify a valid API key for your target cloud team.")
target_data_pool = c_target.pools.find("Name or ID of the target data pool.")
data_source_name = "Name of the data connection in the target data pool the target data model should refer to. Indicate an empty string to point to the global scope."

2020-02-13 16:05:17 - pycelonis: Login successful! Hello s.matthaei@celonis.com


**Create the data model and add the tables.**

In [40]:
if source_data_model.name in target_data_pool.datamodels.names:
    print('A data model with the same name does already exist in the target data pool. Please rename one.')
else: 
    target_data_model = target_data_pool.create_datamodel(source_data_model.name)                         # create target data model

    connection = target_data_pool.data_connections.names[data_source_name] if data_source_name else None  # choose the connection for the data model 

    target_data_model.add_tables_from_pool(source_data_model.tables, connection)                          # add the tables from the connection to the data model

**Copy Activity & Case table settings. If you copy to a data pool that does not contain the case and activity table in the specified connection, this step cannot be performed. Just continue with the next one.**

In [41]:
target_data_model.create_process_configuration(
            activity_table=source_data_model.process_configurations[0].activity_table.data["name"] if source_data_model.process_configurations[0].activity_table else None,
            case_table=source_data_model.process_configurations[0].case_table.data["name"] if source_data_model.process_configurations[0].case_table else None,
            case_column=source_data_model.process_configurations[0].case_column if source_data_model.process_configurations[0].activity_table else None,
            activity_column=source_data_model.process_configurations[0].activity_column if source_data_model.process_configurations[0].activity_table else None,
            timestamp_column=source_data_model.process_configurations[0].timestamp_column if source_data_model.process_configurations[0].activity_table else None,
            sorting_column=source_data_model.process_configurations[0].sorting_column if source_data_model.process_configurations[0].activity_table else None)

[<DatamodelProcessConfiguration, id 6ac4ba6a-1996-4088-8211-7970aa108669, name Unnamed object with ID: 6ac4ba6a-1996-4088-8211-7970aa108669>,]

**Add forgein key relationships.**

In [42]:
for fk in source_data_model.foreign_keys:
    target_data_model.create_foreign_key(fk["source_table"], fk["target_table"], fk["columns"], from_scratch=True)
print('Relationships copied.')

Relationships copied.


**Add table aliases.**

In [38]:
for t in source_data_model.tables:
    if t.alias == t.name and isinstance(source_data_model, pycelonis.objects_ibc.Datamodel):
        target_t = target_data_model.tables.find(t.data["name"])
        try:
            target_t.alias = t.name
        except ValueError:
            pass
    if t.alias != t.name and isinstance(source_data_model, pycelonis.objects_cpm4.Datamodel):
        target_t = target_data_model.tables.find(t.data["name"])
        try:
            target_t.alias = t.alias
        except ValueError:
            pass
print("Please ignore the warnings. Congrats you copied the data model '"+ target_data_model.name + "'!")

Please ignore the warnings. Congrats you copied the data model 'SAP ECC - Accounts Payable Data Model'!
